In [1]:
import numpy as np
from scipy.optimize import minimize

# Constants
pipe_lengths = [100, 150, 200, 100]  # in meters
initial_flows = [50, -30, -20, 0]    # L/s (Initial guess)
diameters = [0.3, 0.35, 0.4, 0.3]    # Initial diameters (meters)
roughness = 0.02                     # Roughness factor (constant)
demand = [0, 50, 40, 0]              # Demand at junctions
cost_constant = 1000                 # Cost factor $1000 per m^3 pipe per meter

# Darcy-Weisbach head loss function
def head_loss(flow, length, diameter, roughness, min_flow=1e-6):
    # Prevent divide by zero by assigning a very small flow if flow is zero
    if abs(flow) < min_flow:
        flow = min_flow
    K = length / (diameter**5) * roughness
    return K * (flow ** 2)

# Hardy Cross correction function to balance flows
def hardy_cross_correction(flows, lengths, diameters, roughness, tolerance=1e-6):
    head_losses = [head_loss(flow, length, diameter, roughness)
                   for flow, length, diameter in zip(flows, lengths, diameters)]
    total_head_loss = sum(head_losses)

    # Check if sum of flows is too small, avoid division by zero
    flow_sum = sum(flows)
    if abs(flow_sum) < tolerance:
        return flows, 0  # No correction needed if flows are zero or nearly zero

    # Apply the Hardy Cross correction
    delta_Q = - total_head_loss / (2 * flow_sum)
    corrected_flows = [flow + delta_Q for flow in flows]

    return corrected_flows, delta_Q

# Pressure calculation using head loss and flow balances
def calculate_pressures(pressure_guess, demand, lengths, diameters, roughness, tolerance=1e-3):
    def residual(pressures):
        flows = np.zeros(len(lengths))
        flows[0] = (pressures[0] - pressures[1]) / head_loss(initial_flows[0], lengths[0], diameters[0], roughness)
        flows[1] = (pressures[1] - pressures[2]) / head_loss(initial_flows[1], lengths[1], diameters[1], roughness)
        flows[2] = (pressures[2] - pressures[3]) / head_loss(initial_flows[2], lengths[2], diameters[2], roughness)
        # Handle case where initial_flows[3] is zero by using a small flow value
        if initial_flows[3] == 0:
            flows[3] = (pressures[3] - pressures[0]) / head_loss(1e-6, lengths[3], diameters[3], roughness)
        else:
            flows[3] = (pressures[3] - pressures[0]) / head_loss(initial_flows[3], lengths[3], diameters[3], roughness)
        return demand - flows

    pressures = np.array(pressure_guess, dtype=np.float64)  # Ensure pressures are float
    for _ in range(20):
        res = residual(pressures)
        if np.max(np.abs(res)) < tolerance:
            break
        pressures -= 0.1 * res  # Iteratively update pressures
    return pressures

# Cost optimization for pipe diameters (quadratic cost function)
def optimize_pipe_cost(lengths, diameters):
    def cost_function(diameters):
        return sum(cost_constant * (d ** 2) * length for d, length in zip(diameters, lengths))

    bounds = [(0.1, 1.0)] * len(diameters)  # Diameter bounds

    # Solve the optimization problem using 'minimize'
    result = minimize(cost_function, diameters, bounds=bounds)
    return result.x  # Return optimized diameters

# Main function to run the simulation
def water_distribution_simulation(initial_flows, pipe_lengths, diameters, roughness, demand):
    # Step 1: Flow balancing with Hardy Cross
    flows = initial_flows
    for i in range(10):  # 10 iterations of Hardy Cross
        flows, delta_Q = hardy_cross_correction(flows, pipe_lengths, diameters, roughness)
        if abs(delta_Q) < 1e-3:
            break

    print(f"Balanced flows after Hardy Cross: {flows}")

    # Step 2: Pressure calculation based on head loss
    pressure_guess = [50, 45, 40, 35]  # Initial guess for pressures at nodes
    pressures = calculate_pressures(pressure_guess, demand, pipe_lengths, diameters, roughness)
    print(f"Calculated pressures at junctions: {pressures}")

    # Step 3: Pipe cost optimization
    optimized_diameters = optimize_pipe_cost(pipe_lengths, diameters)
    print(f"Optimized pipe diameters: {optimized_diameters}")

    return flows, pressures, optimized_diameters

# Run the simulation
final_flows, final_pressures, final_diameters = water_distribution_simulation(initial_flows, pipe_lengths, diameters, roughness, demand)

# Output results
print("\nFinal Flows (L/s):", final_flows)
print("Final Pressures (m):", final_pressures)


Balanced flows after Hardy Cross: [50, -30, -20, 0]
Calculated pressures at junctions: [ 2.48702847e+118 -6.21757117e+133  3.88347564e+148 -7.37253579e+162]
Optimized pipe diameters: [0.1 0.1 0.1 0.1]

Final Flows (L/s): [50, -30, -20, 0]
Final Pressures (m): [ 2.48702847e+118 -6.21757117e+133  3.88347564e+148 -7.37253579e+162]
